In [25]:
## Instalações

# Acessar os valores das ações, petróleo (brent), ibovespa e minério de ferro
!pip install yfinance

# Acessar a variável de entrada do fechamento do dólar
!pip install python-bcb==0.3.3

In [26]:
## Bibliotecas

# Pre Processamento dos dados
import pandas as pd
import numpy as np

# Download das Ações
import yfinance as yf
from bcb import sgs

# Montagem dos Dados
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Baixar os Dados já montados
import joblib
import os

In [27]:
## Definição de Variáveis fixas

acao = ''
start = '2016-01-01'
end = '2025-06-25'

ticker = f'{acao}.SA'

In [28]:
## Definição das Features fixas

# Dólar
dolar = sgs.get({'Close_Dolar': 1}, start=start, end=end).reset_index() # Faz com que Date deixe de ser index e passe a ser uma coluna junto ao Close_Dolar
dolar['Date'] = pd.to_datetime(dolar['Date']) # Muda o type da coluna Date para Datetime para ficar igual as demais variáveis
dolar.set_index('Date', inplace=True) # Retorna Date para index, pois é assim que as demais variáveis estão

# Petróleo (WTI) pelo Yahoo Finance
petroleo = yf.download('CL=F', start=start, end=end)[['Close']] # Preço do barril de petróleo WTI
petroleo.columns = ['Close_Petroleo']

# Ibovespa
ibov = yf.download('^BVSP', start=start, end=end)[['Close']] # Índice ibovespa
ibov.columns = ['Close_Ibovespa']

# Minério de Ferro (supondo índice internacional)
minerio = yf.download('RIO', start=start, end=end)[['Close']] # Ação da mineradora Rio Tinto
minerio.columns = ['Close_Minerio']

/tmp/ipython-input-3893755857.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  petroleo = yf.download('CL=F', start=start, end=end)[['Close']] # Preço do barril de petróleo WTI
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3893755857.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ibov = yf.download('^BVSP', start=start, end=end)[['Close']] # Índice ibovespa
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3893755857.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  minerio = yf.download('RIO', start=start, end=end)[['Close']] # Ação da mineradora Rio Tinto
[*********************100%***********************]  1 of 1 completed


In [29]:
## Funções

def baixar_dados_acao(acao, start, end):
    df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
    df.columns = [f'Close_{acao}', f'Volume_{acao}']
    return df

def preparar_dataframe_completo(df_acao, dolar, petroleo, ibov, minerio):
    df = df_acao.copy()
    df = df.merge(dolar, left_index=True, right_index=True, how='left')
    df = df.merge(petroleo, left_index=True, right_index=True, how='left')
    df = df.merge(ibov, left_index=True, right_index=True, how='left')
    df = df.merge(minerio, left_index=True, right_index=True, how='left')
    df.dropna(inplace=True)
    return df

def normalizar_e_salvar(df, acao):
    scaler = MinMaxScaler()
    dados_escalados = scaler.fit_transform(df)
    joblib.dump(scaler, f'./{acao}/scaler_{acao}.pkl')
    return dados_escalados, scaler

def createXY(dataset, n_past):
    dataX, dataY = [], []
    for i in range(n_past, len(dataset)):
        dataX.append(dataset[i - n_past:i, :])
        dataY.append(dataset[i, 0])
    return np.array(dataX), np.array(dataY)

def gerar_janelas_e_salvar(dados_escalados, df, acao, janela=21):
    X, y = createXY(dados_escalados, janela)
    y_real = df.iloc[janela:][f'Close_{acao}'].values.reshape(-1, 1)

    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    y_real_train, y_real_test = y_real[:split], y_real[split:]

    np.save(f'./{acao}/X_train_{acao}.npy', X_train)
    np.save(f'./{acao}/y_train_{acao}.npy', y_train)
    np.save(f'./{acao}/X_test_{acao}.npy', X_test)
    np.save(f'./{acao}/y_test_{acao}.npy', y_test)
    np.save(f'./{acao}/y_real_{acao}.npy', y_real_test)

## AÇÃO DA ABEV3


In [30]:
acao = 'ABEV3'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA B3SA3

In [31]:
acao = 'B3SA3'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA BBAS3

In [32]:
acao = 'BBAS3'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA BBDC4

In [33]:
acao = 'BBDC4'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA ELET3

In [34]:
acao = 'ELET3'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA ITUB4

In [35]:
acao = 'ITUB4'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA PETR3

In [36]:
acao = 'PETR3'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA PETR4

In [37]:
acao = 'PETR4'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA SBSP3

In [38]:
acao = 'SBSP3'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA VALE3

In [39]:
acao = 'VALE3'

os.makedirs(f'{acao}', exist_ok=True)

abev = baixar_dados_acao(acao, start, end)
df_abev3 = preparar_dataframe_completo(abev, dolar, petroleo, ibov, minerio)
dados_escalados, scaler = normalizar_e_salvar(df_abev3, acao)
gerar_janelas_e_salvar(dados_escalados, df_abev3, acao)

/tmp/ipython-input-3925094849.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed
